# Pandas

## Series

**Series (1D)** – это проиндексированный одномерный массив значений. Он похож на простой словарь типа dict, где имя элемента соответствует ключу, а значение – значению записи.

Объект Series можно создать с помощью конструктора Series, принимающего список значений и (дополнительно) список ключей. Если список ключей не указан, ключами станут индексы исходного массива.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
s = pd.Series(np.random.randn(5)) # Генерация списка из 5 случайных значений
s

In [ ]:
s = pd.Series(np.random.randn(5), # Генерация списка из 5 случайных значений
              index=['a', 'b', 'c', 'd', 'e']) # Список ключей
s

Также можно создать такой объект из словаря: 

In [ ]:
s_new = pd.Series({'a' : -1, 'g' : 3})
s_new

Добавить в Series элемент можно, присвоив новое значение по ключу (как в словаре). Изменить значение можно так же:

In [ ]:
s['f'] = 2 # Добавление новой пары ключ-значение
s['a'] = -10 # Изменение значения для старого ключа
s

Удалить элемент можно с помощью метода **drop**, который принимает как параметры массив с ключами, которые нужно удалить. Метод **drop** по умолчанию не является inplace-методом -- создавая и возвращая новую табличку без указанного элемента, он не меняет старую.

In [ ]:
s.drop(['d', 'b'])
s

Чтобы метод **drop** изменил исходную табличку, необходимо прописать атрибут **inplace=True**. Вот так:

In [ ]:
s.drop(['d', 'b'], inplace=True)
s

Списки Series можно объединять с помощью метода **append**:

In [ ]:
s = s.append(s_new)
s

Как можно заметить, в списке Series ключи не обязательно различны. При попытке изменить значение для ключа, встречающегося в списке неоднократно, меняется значение для *всех* таких ключей.

In [ ]:
s['a'] = 100
s

In [ ]:
s['a'] += 15
s

Списки Series можно сортировать как по ключам, так и по значениям с помощью методов **sort_values** и **sort_index**.

In [ ]:
s = s.sort_values()
s

In [ ]:
s = s.sort_index()
s

Также к спискам Series можно применять арифметические операции, которые выполняются поэлементно. Так, можно сложить два списка одинакового размера, умножить список на число и т.д.

In [ ]:
s * s

In [ ]:
s + 2

## DataFrame

** DataFrame (2D) ** — это проиндексированный двумерный массив значений, соответственно каждый столбец **DataFrame** является структурой **Series**. Он отлично подходит для представления реальных данных: столбцы соответствуют признакам, а  строки - признаковым описаниям отдельных объектов.
### Создание
DataFrame, как и Series, можно создать из словаря:

In [ ]:
df = pd.DataFrame({'numbers' : range(1, 6), 'chars' : ['a'] * 5})
df

Или из двумерного массива. Существует специальный атрибут **columns**, позволяющий прописать заголовки. Если не воспользоваться им, столбцы будут просто пронумерованы.

In [ ]:
df = pd.DataFrame([['a', 1], ['b', 2], ['c', 3]], columns=['chars', 'numbers'])
df

Однако в большинстве случаев удобнее считывать данные из файла. Это делается с помощью метода **read_csv**. Параметром ему передается путь к файлу, с помощью атрибутов указываются номер строки с заголовками (если они есть) и используемый разделитель.

In [ ]:
df = pd.read_csv("df.txt", header=0, sep='\t') # Здесь заголовки лежат в первой строке, а разделителем является табуляция
df

### Добавление данных

В DataFrame можно добавить строку, воспользовавшись методом **append**. Если атрибут **ignore_index** = True, то при добавлении новых записей их индексы не будут учитываться. Так как у строки нет индекса, то при ее добавлении *необходимо* прописать ignore_index=True.

Прибавим строку, заданную словарем:

In [ ]:
line_1 = {'number' : '3', 'city' : 'Arzamas'}
df = df.append(line_1, ignore_index=True)
df

Это же метод позволяет прибавить и другой DataFrame:

In [ ]:
df_2 = pd.DataFrame([[np.nan, 'Belomorsk', 9861], [5, 'Odessa', 1010783]],
                      columns=['number', 'city', 'population'])
df = df.append(df_2, ignore_index=True)
df

Добавление столбцов происходит так же, как и в словарях. Ключом служит имя столбца, значением -- список, который мы добавляем.

In [ ]:
df['country'] = ['Russia', 'Finland', 'Russia', 'Russia', 'Ukraine']
df

Добавить новый столбец можно и по-другому, используя метод **insert**. Первый параметр указывает, после какого столбца следует добавить столбец, второй параметр - это название нового столбца, а третий - добавляемый список.

In [ ]:
is_capital = [True] * 2 + [False] * 3
df.insert(2, 'is_capital', is_capital)
df

### Удаление данных

Для удаления строк и столбцов из DataFrame, как и для Series, существует метод **drop**. Как параметр ему нужно передать список названий строк/столбцов и прописать атрибут оси **axis**, равный 0 для строк и 1 для столбцов. Также у drop есть атрибут **inplace**, с которым мы уже сталкивались.

In [ ]:
df.drop([3], axis=0)

In [ ]:
df.drop(['number'], axis=1, inplace=True)
df

### Объединение таблиц

Для того, чтобы объединить две таблицы, существуют функции **merge** и **concat**.

In [ ]:
df3 = pd.read_csv("df3.txt", header=0, sep='\t')
df3

В качестве параметров функции **merge** подаются две таблицы, для которых можно указать список тех столбцов, которые должны оказаться в новой таблице. Атрибут **on** нужен для обозначения списка столбцов-ключей. Также существует атрибут **how**, который определяет, каким образом будут сливаться таблицы. Ниже представлены его возможные значения и соответственно строки, которые окажутся в новой таблице:
* 'inner': значение строки в столбце-ключе встречается в обеих таблицах. 
* 'outer': значение строки в столбце-ключе встречается хотя бы в одной таблице. 
* 'left': значение строки в столбце-ключе встречается в левой таблице.
* 'right': значение строки в столбце-ключе встречается в правой таблице.

In [ ]:
pd.merge(df, df3[['mayor', 'city']], how='inner', on='city') # Возьмем все столбцы из первой таблицы,
                                                            # добавим столбец с мэрами и городами из второй. 
                                                            # Пересечем таблицы по названию городов

In [ ]:
pd.merge(df, df3, how='left', on='city') # Cделаем то же самое, но из второй таблицы будут добавлены все столбцы.
                                        # Названия городов возьмем из первой таблицы

Функция **concat** принимает массив из объектов, которые следует объединить, и имеет атрибут **axis** - по какой из осей нужно соединять таблицы. 

In [ ]:
pd.concat([df, df3], axis=0)

In [ ]:
pd.concat([df, df3[['mayor']]], axis=1)

Она полезна в том случае, когда в таблицах полное соответствие строк (столбцов) по данной оси. Обратите внимание, что в качестве мэра Арзамаса в табличке выше указан Слепцов (мэр Ярославля). 

### Индексация

Чтобы воспользоваться столбцом как объектом типа Series, нужно обратиться к нему по имени через точку:

In [ ]:
df.city

Также можно представить его как отдельный DataFrame с помощью двойных квадратных скобочек:

In [ ]:
df[['city']]

Таким образом можно вывести DataFrame с произвольным количеством столбцов, просто перечислив их имена в списке:

In [ ]:
df[['city', 'country', 'is_capital']]

Со строками работать не труднее. К ним нельзя обратиться по индексу, но можно брать срезы, как в обычных списках.

In [ ]:
df[2:4]

Для получения первых строк также существует метод **head**:

In [ ]:
df.head(2)

Чтобы выбрать какой-то произвольный список строк и столбцов, можно использовать один из двух методов: **loc** или **iloc**.

Команда **loc** позволяет индексировать DataFrame с помощью обращения к названию осей. При этом *учитывается и начало, и конец среза*. 

In [ ]:
df.loc[1:3, 'city':'population']

Команда **iloc** принимает индексы начала и конца среза, и *конец среза не учитывается*. 

In [ ]:
df.iloc[1:3, 0:2]

Этим методам можно подавать на вход не только срезы, но и списки нужных названий или индексов:

In [ ]:
df.loc[[0, 2, 4], ['city', 'is_capital', 'country']]

### Применение функций к ячейкам, столбцам и строкам

Pandas позволяет применять функции к столбцу с помощью метода **apply**:

In [ ]:
df[['population']].apply(lambda x : x // 2) # Уменьшим численность населения в два раза

In [ ]:
df.apply(max) # Найдем максимум для каждого столбца

То же самое можно делать и со строками, указав атрибут **axis**=1.

Метод **map** можно использовать, чтобы применить функцию к каждой ячейке столбца. Например, с помощью map и словаря можно изменить значения:

In [ ]:
print(df.is_capital)
d = {True : 'Yes', False : 'No'}
df['is_capital'].map(d) # Изменим булевы значения на соответствующие строки в столбце is_capital

Заполнить пустые ячейки может метод **fillna**:

In [ ]:
df.fillna(0)

Изменить тип колонки можно с помощью метода **astype**:

In [ ]:
df['is_capital'].astype('int')

### Сортировки

Отсортировать таблицу по значениям столбца или строки можно, использовав метод **sort_values**. У него есть параметр **ascending**, равный True для сортировки по возрастанию и False - по убыванию, и атрибут **axis**, обозначающий ось.

In [ ]:
df.sort_values('population', ascending=False)

### Фильтрация и селекция

Pandas позволяет фильтровать данные в таблице с помощью логических операторов (&, |, ==, != и так далее). Для этого следует выбрать нужную часть таблицы и применить к ней логическое выражение. Например, вычислим количество городов, население которых превышает 10 000.

In [ ]:
print(len(df[df.population > 1e4])) # Выбираем столбец с населением, сравниваем с 10000,
                                    # выбираем строки, где это правда, и считаем их количество

Или, например, в данной таблице можно отфильтровать данные, оставив информацию только про города, не являющиеся столицами. Для этого нужно в квадратных скобках написать соответствующее логическое выражение. 

In [ ]:
df[df.is_capital == False]

Выражение можно усложнить, выбрав, например, все российские города, не являющиеся столицами.

In [ ]:
df[(df.is_capital == False) & (df.country == 'Russia')]

### Группировка

Pandas позволяет группировать данные с помощью метода **groupby**, принимающего на вход имя признака и возвращающая  объект **DataFrameGroupBy**, в котором хранятся сведения о группировке данных.

In [ ]:
df.groupby('country')

В этом объекте можно выбрать интересующие столбцы по названию (но можно и не выбирать) и применить ряд функций, позволяющих проанализировать сведения о каждой группе. Вот часть таких функций:

![alternate text](https://pp.userapi.com/c841632/v841632172/150fd/DtGqi1-LYig.jpg)

Также можно посмотреть количество различных значений поля, по которому происходит группировка, с помощью функции **size**:

In [ ]:
df.groupby('country').size()

Как пример, рассчитаем суммарное население в каждой стране, представленной в таблице:

In [ ]:
df.groupby('country')['population'].sum()

Функция **get_group** дает возможность посмотреть все ячейки конкретной группы по набору столбцов.

In [ ]:
df.groupby(['country'])[['city', 'is_capital']].get_group('Russia')

### Сводные таблицы

Сводные таблицы помогают обобщать численные данные. Они могут автоматически сортировать и подсчитывать данные, а также вычислять общее или среднее значение в одной таблице. 

В Pandas для этого существует функция **pivot_table**, принимающая на вход список столбцов, по которым будет считаться агрегированные значения, и список столбцов, которые будут строками итоговой таблицы. Атрибут **aggfunc** задает функцию, которая используется для агрегации, а **fill_value** - параметр для замены пустых значений на 0.

In [ ]:
df['living_wage'] = [15307, 84925, 8082, 13730, 3958] # Добавим к таблице еще один столбец
df

In [ ]:
df.pivot_table(['population', 'living_wage'], ['country'], aggfunc='mean', fill_value=0) #Посчитаем средние значения 
# прожиточного минимума и численности населения для разных стран, все пустые ячейки заполним 0

# Пример

Есть две таблички. В первой (data.csv) лежит информация о количестве бюджетных и платных мест по разным специальностям двух факультетов. Во второй (cost.csv) -- информация о стоимости обучения на всех направлениях подготовки ВШЭ. Посчитаем количество денег, полученных за обучение на платной основе по каждому из представленных в первой таблице факультетов. 

In [ ]:
data = pd.read_csv("data.csv", sep=',', header=0) # Считаем файлы
data

In [ ]:
cost = pd.read_csv("cost.csv", sep=',', header=0)
cost.head() # Посмотрим на первые пять строк

Совместим таблички, добавив в первую столбец со стоимостью на соответсвующих направлениях:

In [ ]:
data = pd.merge(data, cost, how='left', on='Направление подготовки') # Ключевой столбец - 'Направление подготовки',
                                                                    # берем строки из первой таблицы
data

In [ ]:
data['Платные места'] = data['Платные места'].astype('int64') # Поменяем тип столбца на int
data['Платные места для иностранцев'] = data['Платные места для иностранцев'].astype('int64')

Добавим к платным местам количество платных мест для иностранцев, и удалим этот столбец. 

In [ ]:
data['Платные места'] += data['Платные места для иностранцев']
data.drop(['Платные места для иностранцев'], axis=1, inplace=True)

In [ ]:
data

Введем новый столбец - количество денег в тысячах рублей, которые получит вуз от абитуриентов за первый год, при условии, что все платные места будут заполнены.

In [ ]:
data['Прибыль'] = data['Платные места'] * data['Стоимость обучения']
data

Теперь построим итоговую сводную таблицу. 

In [ ]:
res = data.pivot_table(['Прибыль'], ['Факультет'], aggfunc='sum', fill_value=0)
res

## Задание

Отсортируйте таблицу data по стоимости обучения.

In [ ]:
# Место для вашего кода

Посчитайте, сколько есть направлений, на которых платных мест строго больше 70.

In [ ]:
# Место для вашего кода

Сгруппируйте направления по факультетам и посчитайте, сколько бюджетных мест на каждом из них.

In [ ]:
# Место для вашего кода